# Prediction of particle size of ZnO

In [ ]:
import pandas as pd
import numpy as numpy

from sklearn.preprocessing import OneHotEncoder
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, root_mean_squared_error, mean_absolute_error

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.lines import Line2D
import shap

# Data preprocessing

In [ ]:
file_path = r"dataset-pathway"
data = pd.read_csv(file_path)
print(data.head())

data.isnull().sum()

data = data.dropna(subset=['Max. size (nm)']) # remove rows with missing data for Max.size
print(f'New rows and columns', data.shape) # check for the new number of rows and columns
data.isnull().sum() # check if all missing values have been removed from 'Max. size'

## Encode categorical features using OHE

ohe = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
encoded_features = ohe.fit_transform(data[['Method category', 'Solvent type']])
encoded_df = pd.DataFrame(encoded_features, columns=ohe.get_feature_names_out(['Method category', 'Solvent type']))
data = pd.concat([data.reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1) # concatenate using reset_index(drop=True)

encoded_columns = ['Method category', 'Solvent type'] # Drop original categorical columns
data = data.drop(columns=encoded_columns)
print(data)

## Impute missing data

exclude_column = ['Max. size (nm)']
columns_to_impute = [col for col in data.columns if col not in exclude_column]
categorical_features = data[columns_to_impute].select_dtypes(include=['object']).columns
numerical_features = data[columns_to_impute].select_dtypes(include=['number']).columns

imputer = IterativeImputer(max_iter=10, random_state=0, verbose=2)
ohe_columns = encoded_df.columns  # Columns created by OneHotEncoder
data[ohe_columns] = imputer.fit_transform(data[ohe_columns])

data[numerical_features] = imputer.fit_transform(data[numerical_features])

# combine the imputed numerical and categorical data

imputed_data = pd.concat([data[ohe_columns], data[numerical_features], data[['Max. size (nm)']]], axis=1)
imputed_data.isnull().sum()

# Feature scaling

numerical_features = data.select_dtypes(include='number').drop(columns=['Max. size (nm)'])
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(numerical_features)

data_scaled = data.copy()
data_scaled[numerical_features.columns] = scaled_features
data = data_scaled
final_data = data
print(final_data)

# Train model - 30 times

In [ ]:

def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Store all results
results = []
best_rmse = np.inf

# Run model 30 times
for seed in range(30):
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

    # Train model
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Predict both train and test
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Calculate metrics
    train_r2 = r2_score(y_train, y_train_pred)
    train_rmse = root_mean_squared_error(y_train, y_train_pred)
    train_mae = mean_absolute_error(y_train, y_train_pred)

    test_r2 = r2_score(y_test, y_test_pred)
    test_rmse = root_mean_squared_error(y_test, y_test_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)

    # Store metrics
    results.append({
        'seed': seed,
        'Train_R2': train_r2, 'Train_RMSE': train_rmse, 'Train_MAE': train_mae,
        'Test_R2': test_r2, 'Test_RMSE': test_rmse, 'Test_MAE': test_mae
    })

    # Track best model based on test RMSE
    if test_rmse < best_rmse:
        best_rmse = test_rmse
        best_seed = seed
        best_model = model
        best_y_train = y_train.copy()
        best_y_train_pred = y_train_pred.copy()
        best_y_test = y_test.copy()
        best_y_test_pred = y_test_pred.copy()

    print(f"Seed {seed} - Train RMSE: {train_rmse:.3f}, Test RMSE: {test_rmse:.3f}")

# Save results
df_results = pd.DataFrame(results)
df_results.to_csv("RF_30seeds_metrics_train_test.csv", index=False)

# Summary
print("\nSummary of metrics over 30 runs:")
print(df_results.describe())


In [ ]:
# Calculate residuals
train_residuals = best_y_train - best_y_train_pred
test_residuals = best_y_test - best_y_test_pred

# Combine for one plot
plt.figure(figsize=(8, 6))

# Plot train residuals
plt.scatter(best_y_train_pred, train_residuals, color='blue', alpha=0.5, edgecolor='k', label='Train')

# Plot test residuals
plt.scatter(best_y_test_pred, test_residuals, color='green', alpha=0.6, edgecolor='k', label='Test')

# Horizontal zero line
plt.axhline(0, color='red', linestyle='--', linewidth=2, label='Zero error')

# Labels and title
plt.xlabel('Predicted Values', fontsize=14, fontweight='bold')
plt.ylabel('Residuals', fontsize=14, fontweight='bold')
plt.title(f'Residuals Plot', fontsize=14, fontweight='bold')
plt.legend(title_fontsize=13, fontsize=14, loc='best', frameon=True)
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Prediction error plot

# Combine train and test into one for cleaner plot (optional)
y_all_actual = np.concatenate([y_train, y_test])
y_all_pred = np.concatenate([y_train_pred, y_pred])
labels = ['Train'] * len(y_train) + ['Test'] * len(y_test)

plt.figure(figsize=(8, 6))

# Scatterplot with regression line
sns.scatterplot(x=y_all_pred, y=y_all_actual, hue=labels, alpha=0.6, edgecolor='k')
sns.regplot(x=y_all_pred, y=y_all_actual, scatter=False, color='black', line_kws={"label": "Best Fit Line"})

# Identity line (ideal prediction)
min_val = min(y_all_actual.min(), y_all_pred.min())
max_val = max(y_all_actual.max(), y_all_pred.max())
plt.plot([min_val, max_val], [min_val, max_val], 'r--', label='Ideal (y = x)')

# Labels and legend
plt.xlabel('Predicted Values')
plt.ylabel('Actual Values')
plt.title('Prediction Error Plot with Identity and Best Fit Lines')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# SHAP

In [ ]:
# Create a SHAP explainer object
explainer = shap.Explainer(model, X_train)

# Calculate SHAP values
#shap_values = explainer(X_test)  # Use test data to explain predictions
shap_values = explainer(X_test, check_additivity=False)

plt.figure(figsize=(16, 16))
shap.summary_plot(shap_values, X_test, plot_type="bar")

# Convert the SHAP values to a Pandas DataFrame for easy manipulation
shap_df = pd.DataFrame(shap_values.values, columns=X.columns)

# Calculate the mean absolute SHAP value for each feature
mean_abs_shap = shap_df.abs().mean(axis=0)

# Sort the features based on their mean absolute SHAP value
top_10_features = mean_abs_shap.sort_values(ascending=False).head(10)

# Print the top 10 features
print(top_10_features)

# Plot the top 10 features using a bar plot
plt.figure(figsize=(10, 6))
top_10_features.plot(kind='bar', color='skyblue')
#plt.title('Top 10 Features Based on Mean Absolute SHAP Values')
plt.xlabel('Features', fontsize=12, fontweight='bold')
plt.ylabel('Mean Absolute SHAP Value', fontsize=12, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()